# 第15章 类型提示进阶

## 15.1 重载的签名

@typing.overload装饰器用于函数的重载。

In [1]:
import functools
import operator
from collections.abc import Iterable
from typing import overload, Union, TypeVar

T = TypeVar('T')
S = TypeVar('S')  # <1>

@overload
def sum(it: Iterable[T]) -> Union[T, int]: ...  # <2>
@overload
def sum(it: Iterable[T], /, start: S) -> Union[T, S]: ...  # <3>
def sum(it, /, start=0):  # <4>
    return functools.reduce(operator.add, it, start)

其中：
- 省略号（...）没有特殊作用，只是为了满足句法的要求，类似于pass。
- 参数`/`用于分隔不同类型的参数，比如位置参数和关键字参数。

## 15.2 TypedDict

处理动态数据结构时，容易误用TypedDict来避免错误。

TypedDict的作用：
- 使用与类相似的句法注解字典，为各个“字段”的值提供类型提示。
- 通过一个构造函数告诉类型检查工具，字典应具有指定的键和指定类型的值。

In [2]:
from typing import TypedDict, List
import json

class BookDict(TypedDict):
    isbn: str
    title: str
    authors: List[str]
    pagecount: int

In [5]:
AUTHOR_ELEMENT = '<AUTHOR>{}</AUTHOR>'

def to_xml(book: BookDict) -> str:
    elements: List[str] = []  
    for key, value in book.items():
        if isinstance(value, list):  
            elements.extend(AUTHOR_ELEMENT.format(n)
                for n in value)
        else:
            tag = key.upper()
            elements.append(f'<{tag}>{value}</{tag}>')
    xml = '\n\t'.join(elements)
    return f'<BOOK>\n\t{xml}\n</BOOK>'

In [6]:
def from_json(data: str) -> BookDict:
    whatever = json.loads(data)
    return whatever

In [9]:
print(to_xml({
        'isbn': '0134757599',
        'title': 'Refactoring, 2e',
        'authors': ['Martin Fowler', 'Kent Beck'],
        'pagecount': 478,
    }))

<BOOK>
	<ISBN>0134757599</ISBN>
	<TITLE>Refactoring, 2e</TITLE>
	<AUTHOR>Martin Fowler</AUTHOR>
	<AUTHOR>Kent Beck</AUTHOR>
	<PAGECOUNT>478</PAGECOUNT>
</BOOK>


## 15.3 类型校正

typing.cast可用于处理不受控制的代码中存在的类型检查问题或不正确的类型提示。

In [10]:
from typing import cast

def find_first_str(a: list[object]) -> str:
    index = next(i for i, x in enumerate(a) if isinstance(x, str))
    # 至少有一个字符串才能执行到这里
    return cast(str, a[index])

## 15.4 在运行时读取类型提示

类型提示使用量增加会引起两个问题：
- 如果类型提示很多，导入模块使用的CPU和内存会更多。
- 引用尚未定义的类型需要使用字符串，而不是真正的类型。

问题解决方案：
- 不要直接读取`__annotations__`属性，使用`inspect.get_annotations`或`typing.get_type_hints`。
- 自己编写一个函数，简单包装`inspect.get_annotations`或`typing.get_type_hints`。在基准代码中调用自定义的函数，当行为有变时，只需修改一个函数即可。

## 15.5 实现一个泛化类

In [2]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):         
        """从可迭代对象中添加元素"""

    @abc.abstractmethod
    def pick(self):  # <3>
        """随机删除元素，再返回被删除的元素。
        
        如果实例为空，那么这个方法应该抛出LookupError
        """

    def loaded(self):
        """如果至少有一个元素，就返回True，否则返回False"""
        return bool(self.inspect())

    def inspect(self):
        """返回由容器中的当前元素构成的有序元组"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  
        return tuple(items)

In [4]:
import random

from collections.abc import Iterable
from typing import TypeVar, Generic

T = TypeVar('T')

class LottoBlower(Tombola, Generic[T]):

    def __init__(self, items: Iterable[T]) -> None: 
        self._balls = list[T](items)

    def load(self, items: Iterable[T]) -> None:  
        self._balls.extend(items)

    def pick(self) -> T:  
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)

    def loaded(self) -> bool:  
        return bool(self._balls)

    def inspect(self) -> tuple[T, ...]:  
        return tuple(self._balls)

In [5]:
machine = LottoBlower[int](range(1, 11))
first = machine.pick()
remain = machine.inspect()

first, remain

(3, (1, 2, 4, 5, 6, 7, 8, 9, 10))

**泛型基本术语：**

- 泛型：具有一个或多个类型变量的类型。例如：`LottoBlower[T]`。
- 形式类型参数：泛型声明中出现的类型变量。例如：`abc.Mapping[KT, VT]`中的`KT`和`VT`。
- 参数化类型：使用具体类型参数声明的类型。例如：`LottoBlower[int]`和`abc.Mapping[str, float]`。
- 具体类型参数：声明参数化类型时，为参数提供的具体类型。例如：`LottoBlower[int]`中的`int`。

## 15.6 型变

### 15.6.1 协变

In [6]:
from typing import TypeVar, Generic


class Beverage:
    """任何饮料"""


class Juice(Beverage):
    """任何果汁"""


class OrangeJuice(Juice):
    """使用巴西橙子制作的美味果汁"""


# 设置covariant=True，表示协变的类型参数
T_co = TypeVar('T_co', covariant=True)  


class BeverageDispenser(Generic[T_co]):  
    def __init__(self, beverage: T_co) -> None:
        self.beverage = beverage

    def dispense(self) -> T_co:
        return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None: 
    """安装一个果汁自动售货机"""

In [7]:
orange_juice_dispenser = BeverageDispenser(OrangeJuice())
install(orange_juice_dispenser)

### 15.6.2 逆变

In [8]:
from typing import TypeVar, Generic

class Refuse:
    """任何废弃物"""

class Biodegradable(Refuse):
    """可生物降解的废弃物"""

class Compostable(Biodegradable):
    """可制成肥料的废弃物"""

T_contra = TypeVar('T_contra', contravariant=True)  # <2>

class TrashCan(Generic[T_contra]):  # <3>
    def put(self, refuse: T_contra) -> None:
        """在倾倒之前存放垃圾"""

def deploy(trash_can: TrashCan[Biodegradable]):
    """放置一个垃圾桶，存放可生物降解的废弃物"""

In [9]:
bio_can: TrashCan[Biodegradable] = TrashCan()
deploy(bio_can)

In [10]:
trash_can: TrashCan[Refuse] = TrashCan()
deploy(trash_can)

### 15.6.3 型变总结

- 不变类型：如果一个形式类型参数既出现在方法参数的类型提示中，又出现在方法的返回类型中，那么该参数必须是不可变的。

给定两个类型A和B，B与A相容，而且均不是`Any`
- `A:>B`：A是B的超类，或者A与B类型相同。
- `A<:B`：B是A的子类，或者A与B类型相同。

- 协变类型：对于`A:>B`，当满足`C[A] :> C[B]`时，泛型`C`是可协变的。
- 逆变类型：对于`A:>B`，当满足`K[A] <: K[B]`是，泛型`K`是可逆变的。

**型变经验法则：**
- 如果一个形式类型参数定义的是从对象中获取的数据类型，那么该形式类型参数可能是协变的。
- 如果一个形式类型参数定义的是对象初始化之后，向对象输入的数据类型，那么该形式类型参数可能是逆变的。
- 如果一个形式类型参数定义的是从对象中获取的数据类型，同时也是向对象输入的数据类型，那么该形式类型参数必定是不可变的。
- 形式参数类型最好是不变的。

## 15.7 实现泛化静态协议

In [11]:
import math
from typing import NamedTuple, SupportsAbs

class Vector2d(NamedTuple):
    x: float
    y: float

    def __abs__(self) -> float:  # 让Vector2d与SupportsAbs相容
        return math.hypot(self.x, self.y)

def is_unit(v: SupportsAbs[float]) -> bool: 
    """v的模接近1时为True"""
    return math.isclose(abs(v), 1.0)

In [12]:
v0 = Vector2d(0, 1)
sqrt2 = math.sqrt(2)
v1 = Vector2d(sqrt2 / 2, sqrt2 / 2)
v2 = Vector2d(1, 1)
v3 = complex(.5, math.sqrt(3) / 2)
v4 = 1

assert is_unit(v0)
assert is_unit(v1)
assert not is_unit(v2)
assert is_unit(v3)
assert is_unit(v4)

## 15.8 杂谈

- 类型无底洞：使用类型检查工具，有时迫不得已要导入不需要知道的类，除了编写类型提示外，这些类在代码中根本用不到，也没有文档记录。
- 其他语言的型变表示法：
    - 协变或逆变不是类型变量的性质，而是使用类型变量定义的泛化类的性质。
    - 在Python中的类型变量应该仿照Kotlin和C#，使用out修饰符表示T是一种输出类型，使用in修饰符表示T是一种输入类型。